In [2]:
import os
os.chdir(os.path.dirname(os.getcwd()))

In [3]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import matplotlib.pyplot as plt
import joblib
import seaborn as sns
# min max scaling
from sklearn.preprocessing import MinMaxScaler
# standard scaler
from sklearn.preprocessing import StandardScaler

In [4]:
# list paths in 'data/citylearn_challenge_2022_phase_1/' that start with Build
# and end with .csv
files = [f for f in os.listdir('data/citylearn_challenge_2022_phase_1/') if f.startswith('Build') and f.endswith('.csv')]
files

['Building_4.csv',
 'Building_5.csv',
 'Building_1.csv',
 'Building_2.csv',
 'Building_3.csv']

In [5]:
weather = pd.read_csv('data/citylearn_challenge_2022_phase_1/weather.csv')
weather

,Outdoor Drybulb Temperature [C],Relative Humidity [%],Diffuse Solar Radiation [W/m2],Direct Solar Radiation [W/m2],6h Prediction Outdoor Drybulb Temperature [C],12h Prediction Outdoor Drybulb Temperature [C],24h Prediction Outdoor Drybulb Temperature [C],6h Prediction Relative Humidity [%],12h Prediction Relative Humidity [%],24h Prediction Relative Humidity [%],6h Prediction Diffuse Solar Radiation [W/m2],12h Prediction Diffuse Solar Radiation [W/m2],24h Prediction Diffuse Solar Radiation [W/m2],6h Prediction Direct Solar Radiation [W/m2],12h Prediction Direct Solar Radiation [W/m2],24h Prediction Direct Solar Radiation [W/m2]
0,20.0,84.0,0.0,0.0,18.3,22.8,20.0,81.0,68.0,81.0,25.0,964.0,0.0,100.0,815.0,0.0
1,20.1,79.0,0.0,0.0,19.4,22.8,19.4,79.0,71.0,87.0,201.0,966.0,0.0,444.0,747.0,0.0
2,19.7,78.0,0.0,0.0,21.1,22.2,19.4,73.0,73.0,87.0,420.0,683.0,0.0,592.0,291.0,0.0
3,19.3,78.0,0.0,0.0,22.2,22.8,19.4,71.0,71.0,90.0,554.0,522.0,0.0,491.0,153.0,0.0
4,18.9,78.0,0.0,0.0,21.7,22.2,18.9,73.0,71.0,90.0,778.0,444.0,0.0,734.0,174.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,20.6,84.0,26.0,130.0,20.1,19.4,20.6,79.0,79.0,73.0,0.0,201.0,27.0,0.0,444.0,143.0
8756,21.1,81.0,0.0,0.0,19.7,21.1,20.0,78.0,73.0,76.0,0.0,420.0,0.0,0.0,592.0,0.0
8757,21.7,79.0,0.0,0.0,19.3,22.2,20.6,78.0,71.0,70.0,0.0,554.0,0.0,0.0,491.0,0.0
8758,21.3,76.0,0.0,0.0,18.9,21.7,20.6,78.0,73.0,73.0,0.0,778.0,0.0,0.0,734.0,0.0


In [24]:
weather[:50]

,Outdoor Drybulb Temperature [C],Relative Humidity [%],Diffuse Solar Radiation [W/m2],Direct Solar Radiation [W/m2],6h Prediction Outdoor Drybulb Temperature [C],12h Prediction Outdoor Drybulb Temperature [C],24h Prediction Outdoor Drybulb Temperature [C],6h Prediction Relative Humidity [%],12h Prediction Relative Humidity [%],24h Prediction Relative Humidity [%],6h Prediction Diffuse Solar Radiation [W/m2],12h Prediction Diffuse Solar Radiation [W/m2],24h Prediction Diffuse Solar Radiation [W/m2],6h Prediction Direct Solar Radiation [W/m2],12h Prediction Direct Solar Radiation [W/m2],24h Prediction Direct Solar Radiation [W/m2]
0,20.0,84.0,0.0,0.0,18.3,22.8,20.0,81.0,68.0,81.0,25.0,964.0,0.0,100.0,815.0,0.0
1,20.1,79.0,0.0,0.0,19.4,22.8,19.4,79.0,71.0,87.0,201.0,966.0,0.0,444.0,747.0,0.0
2,19.7,78.0,0.0,0.0,21.1,22.2,19.4,73.0,73.0,87.0,420.0,683.0,0.0,592.0,291.0,0.0
3,19.3,78.0,0.0,0.0,22.2,22.8,19.4,71.0,71.0,90.0,554.0,522.0,0.0,491.0,153.0,0.0
4,18.9,78.0,0.0,0.0,21.7,22.2,18.9,73.0,71.0,90.0,778.0,444.0,0.0,734.0,174.0,0.0
5,18.3,81.0,0.0,0.0,22.2,21.1,18.3,71.0,76.0,93.0,903.0,392.0,0.0,811.0,396.0,0.0
6,18.3,81.0,25.0,100.0,22.8,20.6,18.9,68.0,78.0,90.0,964.0,231.0,15.0,815.0,320.0,14.0
7,19.4,79.0,201.0,444.0,22.8,20.6,20.0,71.0,73.0,87.0,966.0,27.0,116.0,747.0,143.0,42.0
8,21.1,73.0,420.0,592.0,22.2,20.0,20.0,73.0,76.0,87.0,683.0,0.0,252.0,291.0,0.0,74.0
9,22.2,71.0,554.0,491.0,22.8,20.6,20.0,71.0,70.0,84.0,522.0,0.0,292.0,153.0,0.0,35.0


In [25]:
# loop through csv files and extract consumption and generation data
all_buildings = pd.DataFrame()
for file in files:
    df = pd.read_csv('data/citylearn_challenge_2022_phase_1/' + file)
    filename = file.split('.')[0].split('_')[1]
    df = pd.concat([df, weather], axis=1)
    df['day_year'] = df.index
    df['number'] = int(filename)
    #df['weekday_hour'] = df['Day Type'].astype(int).astype(str) + "-" +  df['Hour'].astype(int).astype(str)
    df["hour_x"] = np.cos(2*np.pi* df["Hour"] / 24)
    df["hour_y"] = np.sin(2*np.pi* df["Hour"] / 24)
    
    df["month_x"] = np.cos(2*np.pi* df["Month"] / (12))
    df["month_y"] = np.sin(2*np.pi*df["Month"] / (12))

    df['Hour'] = df['Hour'].shift(-1)
    df['dayinyear'] = df['day_year'] // 24 
    # create a timestamp column from the dayinyear and hour from origin 23:00 31/07/2021
    df['timestamp'] = pd.to_datetime(df['dayinyear'], unit='D', origin='2021-07-31 23:00') + pd.to_timedelta(df['Hour'] - 1, unit='h')
    df["Month"] = df['Month'].shift(1)
    df["month_x"] = df["month_x"].shift(1)
    df["month_y"] = df["month_y"].shift(1)
    df["hour_x"] = df["hour_x"].shift(1)
    df["hour_y"] = df["hour_y"].shift(1)
    # target one step agead
    if filename == '4':
        df['net'] = df['Equipment Electric Power [kWh]'] - (df['Solar Generation [W/kW]'] * 5)/1000
    else:
        df['net'] = df['Equipment Electric Power [kWh]'] - (df['Solar Generation [W/kW]'] * 4)/1000
    df['net_target'] = df['net']
    df['net+1'] = df['net'].shift(-1)
    df['net-23'] = df['net'].shift(23)
    
    # rename the equipment electric power column to cons_target
    df = df.rename(columns={'Equipment Electric Power [kWh]': 'cons_target'})
    df.drop(['Daylight Savings Status','DHW Heating [kWh]',
       'Cooling Load [kWh]', 'Heating Load [kWh]', 'Day Type'], axis=1, inplace=True)

    scaler = MinMaxScaler()
    # Fit and apply minmax scaler
    df['net_target'] = scaler.fit_transform(df['net_target'].values.reshape(-1, 1))
    # save scaler
    scaler_name = 'scalers/scaler_net_' + filename + '.save'
    joblib.dump(scaler, scaler_name)
    # Fit and apply minmax scaler
    df['cons_target'] = scaler.fit_transform(df['cons_target'].values.reshape(-1, 1))
    # save scaler
    scaler_name = 'scalers/scaler_cons_' + filename + '.save'
    joblib.dump(scaler, scaler_name)
    # log transform cons_target
    for col in df.columns:
        if 'Radiation' in col or 'radiation' in col:
            df[col] = np.log1p(df[col])

    df['net_target+1'] = df['net_target'].shift(-1)
    df['net_target-23'] = df['net_target'].shift(23)

    df['diffuse_solar_radiation+1'] = df['12h Prediction Diffuse Solar Radiation [W/m2]'].shift(11)
    df['direct_solar_radiation+1'] = df['12h Prediction Direct Solar Radiation [W/m2]'].shift(11)
    df['relative_humidity+1'] = df['12h Prediction Relative Humidity [%]'].shift(11)
    df['drybulb_temp+1'] = df['12h Prediction Outdoor Drybulb Temperature [C]'].shift(11)

    # Remove column names containing the word 'Prediction'
    df = df[[col for col in df.columns if 'Prediction' not in col]]
        
    all_buildings = pd.concat([all_buildings, df], axis=0)
    print(len(all_buildings))
# drop columns with more than 1000 missing values
all_buildings = all_buildings[all_buildings.columns[all_buildings.isnull().mean() < 0.8]]
all_buildings

8760
17520
26280
35040
43800


,Month,Hour,cons_target,Solar Generation [W/kW],Outdoor Drybulb Temperature [C],Relative Humidity [%],Diffuse Solar Radiation [W/m2],Direct Solar Radiation [W/m2],day_year,number,...,net,net_target,net+1,net-23,net_target+1,net_target-23,diffuse_solar_radiation+1,direct_solar_radiation+1,relative_humidity+1,drybulb_temp+1
0,NaN,1.0,0.417676,0.000000,20.0,84.0,0.000000,0.000000,0,4,...,2.819150,0.743323,1.928167,NaN,0.631288,NaN,NaN,NaN,NaN,NaN
1,7.0,2.0,0.285671,0.000000,20.1,79.0,0.000000,0.000000,1,4,...,1.928167,0.631288,0.475817,NaN,0.448667,NaN,NaN,NaN,NaN,NaN
2,8.0,3.0,0.070495,0.000000,19.7,78.0,0.000000,0.000000,2,4,...,0.475817,0.448667,0.458233,NaN,0.446456,NaN,NaN,NaN,NaN,NaN
3,8.0,4.0,0.067890,0.000000,19.3,78.0,0.000000,0.000000,3,4,...,0.458233,0.446456,0.446017,NaN,0.444920,NaN,NaN,NaN,NaN,NaN
4,8.0,5.0,0.066080,0.000000,18.9,78.0,0.000000,0.000000,4,4,...,0.446017,0.444920,0.465883,NaN,0.447418,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,7.0,20.0,0.241327,13.670833,20.6,84.0,3.295837,4.875197,8755,3,...,1.417733,0.591003,1.075767,0.861167,0.547147,0.519626,0.0,0.0,81.0,21.1
8756,7.0,21.0,0.176317,0.000000,21.1,81.0,0.000000,0.000000,8756,3,...,1.075767,0.547147,1.162883,0.966817,0.558319,0.533175,0.0,0.0,79.0,21.7
8757,7.0,22.0,0.190595,0.000000,21.7,79.0,0.000000,0.000000,8757,3,...,1.162883,0.558319,1.107300,0.736717,0.551191,0.503666,0.0,0.0,76.0,21.3
8758,7.0,23.0,0.181485,0.000000,21.3,76.0,0.000000,0.000000,8758,3,...,1.107300,0.551191,0.756533,0.703850,0.506207,0.499451,0.0,0.0,76.0,20.9


In [26]:
all_buildings.dropna(inplace=True)
all_buildings.sort_values(['number', 'day_year'], inplace=True)
all_buildings.reset_index(drop=True, inplace=True)
# remove rows with dayinyear 1 and 365
#all_buildings = all_buildings[all_buildings['dayinyear'] != 1]
#all_buildings = all_buildings[all_buildings['dayinyear'] != 365]
# save building 4 to test and 0,1,2,3 to train
all_buildings[all_buildings['number'] == 4].to_csv('data/extra_test.csv')
all_buildings.to_csv('data/extra_train.csv')

In [9]:
all_buildings[all_buildings['building'] != 4][['Month', 'Hour', 'Day Type', 'day_year','dayinyear','timestamp', 'number']]

,Month,Hour,Day Type,day_year,dayinyear,timestamp,number
2,8.0,0.0,2.0,48,2,2021-08-02 00:00:00,1
3,8.0,1.0,2.0,49,2,2021-08-02 01:00:00,1
4,8.0,2.0,2.0,50,2,2021-08-02 02:00:00,1
5,8.0,3.0,2.0,51,2,2021-08-02 03:00:00,1
6,8.0,4.0,2.0,52,2,2021-08-02 04:00:00,1
...,...,...,...,...,...,...,...
43653,7.0,19.0,7.0,8755,364,2022-07-30 19:00:00,5
43654,7.0,20.0,7.0,8756,364,2022-07-30 20:00:00,5
43655,7.0,21.0,7.0,8757,364,2022-07-30 21:00:00,5
43656,7.0,22.0,7.0,8758,364,2022-07-30 22:00:00,5


In [14]:
all_buildings.loc[all_buildings['number'] == '4']

,Month,Hour,cons_target,Solar Generation [W/kW],Outdoor Drybulb Temperature [C],Relative Humidity [%],Diffuse Solar Radiation [W/m2],Direct Solar Radiation [W/m2],day_year,number,...,net,net_target,net+1,net-23,net_target+1,net_target-23,diffuse_solar_radiation+1,direct_solar_radiation+1,relative_humidity+1,drybulb_temp+1
0,NaN,NaN,0.417676,0.000000,20.0,84.0,0.000000,0.000000,0,4,...,2.819150,0.743323,1.928167,NaN,0.631288,NaN,NaN,NaN,NaN,NaN
1,7.0,24.0,0.285671,0.000000,20.1,79.0,0.000000,0.000000,1,4,...,1.928167,0.631288,0.475817,NaN,0.448667,NaN,NaN,NaN,NaN,NaN
2,8.0,1.0,0.070495,0.000000,19.7,78.0,0.000000,0.000000,2,4,...,0.475817,0.448667,0.458233,NaN,0.446456,NaN,NaN,NaN,NaN,NaN
3,8.0,2.0,0.067890,0.000000,19.3,78.0,0.000000,0.000000,3,4,...,0.458233,0.446456,0.446017,NaN,0.444920,NaN,NaN,NaN,NaN,NaN
4,8.0,3.0,0.066080,0.000000,18.9,78.0,0.000000,0.000000,4,4,...,0.446017,0.444920,0.465883,NaN,0.447418,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,7.0,18.0,0.249649,18.613333,20.6,84.0,3.295837,4.875197,8755,4,...,1.591967,0.589014,0.389433,0.565333,0.437805,0.459923,0.0,0.0,81.0,21.1
8756,7.0,19.0,0.057697,0.000000,21.1,81.0,0.000000,0.000000,8756,4,...,0.389433,0.437805,1.186483,0.630883,0.538028,0.468165,0.0,0.0,79.0,21.7
8757,7.0,20.0,0.175785,0.000000,21.7,79.0,0.000000,0.000000,8757,4,...,1.186483,0.538028,3.155883,2.179383,0.785664,0.662877,0.0,0.0,76.0,21.3
8758,7.0,21.0,0.467565,0.000000,21.3,76.0,0.000000,0.000000,8758,4,...,3.155883,0.785664,2.054717,2.966050,0.647201,0.761794,0.0,0.0,76.0,20.9
